In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data.shape

(50000, 2)

In [4]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [5]:
# positive -> 1
# negative -> 0
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [6]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size = 0.25, random_state=42)
test_data.shape

(12500, 2)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [9]:
X_train

array([[   0,    0,    0, ...,    1,   13,  190],
       [ 148,   48,    1, ...,  103,    3,  577],
       [   0,    0,    0, ...,  292,   29, 2104],
       ...,
       [   0,    0,    0, ..., 1614,    2,  593],
       [   0,    0,    0, ...,  246,  103,  125],
       [   0,    0,    0, ...,   70,   72, 2069]])

In [10]:
X_test.shape

(12500, 200)

In [11]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [12]:
Y_train

27434    0
13400    0
883      0
7303     0
45124    1
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 37500, dtype: int64

In [13]:
# LSTM MODEL BUILDING
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [15]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 173s 361ms/step - accuracy: 0.7234 - loss: 0.5400 - val_accuracy: 0.7861 - val_loss: 0.4510
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 168s 358ms/step - accuracy: 0.8382 - loss: 0.3802 - val_accuracy: 0.8171 - val_loss: 0.3996
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 193s 339ms/step - accuracy: 0.8747 - loss: 0.3154 - val_accuracy: 0.8453 - val_loss: 0.3573
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 158s 337ms/step - accuracy: 0.8812 - loss: 0.2940 - val_accuracy: 0.8541 - val_loss: 0.3596
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 171s 365ms/step - accuracy: 0.8739 - loss: 0.3041 - val_accuracy: 0.8693 - val_loss: 0.3334


In [16]:
loss, accuracy = model.evaluate(X_test, Y_test)
print("Model Accuracy: {:.2f}%".format(accuracy*100))

391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.8776 - loss: 0.3231
Model Accuracy: 87.49%


In [17]:
# Building Predictive System

def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [18]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step


'positive'

In [19]:
predictive_system("Overall long and slow, boring")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


'negative'

In [20]:
model.save("model.h5")

In [21]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [22]:
from keras.models import load_model
import joblib
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = load_model("model.h5")
tokenizer = joblib.load("tokenizer.pkl")

def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [23]:
review_sentiment = predictive_system("Beautiful cinematorgraphy")
review_sentiment

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step


'positive'

In [25]:
pip install gradio

  Using cached gradio-4.38.1-py3-none-any.whl.metadata (15 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached altair-5.3.0-py3-none-any.whl.metadata (9.2 kB)
  Using cached ffmpy-0.3.2.tar.gz (5.5 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached gradio_client-1.1.0-py3-none-any.whl.metadata (7.1 kB)
     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/51.6 kB ? eta -:--:--
     ------------------------------ ------- 41.0/51.6 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 51.6/51.6 kB 530.9 kB/s eta 0:00:00
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Usi

In [26]:
import gradio as gr
title = "MOVIE SENTIMENT ANALYSIS APPLICATION"

app = gr.Interface(fn = predictive_system, inputs="textbox", outputs="textbox", title=title)

app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
